In [23]:
# prompt: 讀取/content/匯出結果.xlsx

import pandas as pd

# Install openpyxl if you haven't already
!pip install openpyxl

df = pd.read_excel('/content/中國地方農民工欠薪治理文件與政策.xlsx', engine='openpyxl')


In [24]:
df

,txt檔名,txt內容
0,240529_南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知.txt,【法宝引证码】 CLI.12.7550354\n原文链接：https://www.p...
1,180208_南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知.txt,【法宝引证码】 CLI.12.1437433\n原文链接：https://www.p...
2,200210_南京市人民政府关于废止南京市建筑业施工企业民工工资保障金管理办法的决定.txt,【法宝引证码】 CLI.12.1437433\n原文链接：https://www.p...
3,221027_h_南京市人力资源和社会保障局关于缓缴农民工工资保证金有关事项的通知.txt,【法宝引证码】 CLI.12.5628237\n原文链接：https://www.p...
4,221206_h_南京市人力资源和社会保障局、南京市城乡建设委员会、南京市住房保障和房产局、...,【法宝引证码】 CLI.12.5684027\n原文链接：https://www.p...
5,230213_h_南京市人力资源和社会保障局、南京市城乡建设委员会、南京市住房保障和房产局、...,【法宝引证码】 CLI.12.6442438\n原文链接：https://www.p...
6,170410_南京市政府办公厅关于全面治理拖欠农民工工资问题的实施意见.txt,【法宝引证码】 CLI.12.1268437\n原文链接：https://www.p...


In [25]:
pip install openai==0.28

In [ ]:
import openai

# Set the OpenAI API key
openai.api_key = ""

import openai
import pandas as pd

# 定義提取關鍵字的函數，允許選擇不同的模型
def extract_keywords(text, model):
    prompt = f"""

---

原始文件内容如下：

{text}

一、任务角色与目标

你是政策研究员，目标是针对每份政策文件：
1. 基本信息
2. 第一层编码：逐项评估政策中各类治理条款的存在与强度（0 = 无；1 = 概括式规定；2 = 具体规定）；
3. 第二层编码：根据整份政策语境，判读其在治理结构中对劳工的政策方向性（-1 / 0 / +1）。

二、Codebook 核心字段

基本信息（请撷取文件中明确信息）：
- ｜省份｜
- ｜地级市｜

第一层编码：制度条款强度（以 0 / 1 / 2 编码）

| 指标代码 | 指标含义 |
|----------|----------|
| Importance | 0 = 一般通知、会议纪要；1 = 明确具规范性或有结构性影响 |
| E_保证金 | 是否提及保证金制度，及其详细程度 |
| E_工资专用账户 | 是否明确设立工资专户制度 |
| E_联合企业惩戒 | 是否建立黑名单或信用联合惩戒机制 |
| G_周转金 | 是否建立工资应急周转金制度 |
| G_工作考核办法 | 是否有对治理欠薪的绩效评估或责任考核制度 |
| J_司法回应性 | 是否设有法律援助、绿色通道、巡回法庭等司法机制 |
| J_欠薪刑罚化 | 是否将恶意欠薪行为纳入刑事处罚 |
| J_恶意讨薪刑罚化 | 是否将工人维权行为（如上访）刑事化或负面化处理 |

分类标准：
- 1 = 概括式规定：只提及制度、方向或责任单位，未说明操作细节。
- 2 = 具体规定：明确指出操作方式、流程、标准或法律后果。
- 所有编码为 1 或 2 的项目，请提供政策文本中的原文句子或关键措辞作为左证。

第二层编码：政策方向性变项（诠释性判读）

| 指标名称 | 编码说明 |
| --- | --- |
| Direction_E | 企业责任要求强度：+1 = 明确强化企业义务；0 = 规定模糊；-1 = 倾向宽容企业或减轻其责任 |
| Direction_G | 政府让步程度：+1 = 设立周转金或责任考核等主动承担机制；0 = 承诺流于形式；-1 = 明确减轻或逃避责任 |
| Direction_J | 司法机制回应性：+1 = 明确提供工人司法保障机制；0 = 模糊或未涉及；-1 = 利用司法对工人维权进行压制 |
| Overall_Direction | 综合三项方向性指标，整体倾向：+1 = 偏向保障劳工；0 = 中性；-1 = 偏向维稳或压制劳工权益 |

补充语义观察

| 指标名称 | 判读说明 |
| --- | --- |
| Mention_Labor_Rights | 是否提及“劳动权益”“合法权益”等关键用语（0 = 无 / 1 = 有） |
| Blame_Workers | 是否出现指责工人言辞（如“恶意上访”“非法聚集”等）（0 = 无 / 1 = 有） |

备注说明

- 所有编码为 1 或 2 的制度条款，请提供政策文本中的原文句子或关键措辞作为左证。
- 第二层方向性中，如编码为 +1 或 -1，请在备注中说明语境、政策措辞或潜台词的解释理由。
- 若政策为机构调整、经办单位转移等行政性事务，未涉及制度内容或方向导向，请整体评为 0。
- output格式不要是table，保持格式內容有："省份：","地级市：","Importance：","E_保证金：","E_工资专用账户：","E_联合企业惩戒：","G_周转金：","G_工作考核办法："，
    "J_司法回应性："，
    "J_欠薪刑罚化："，
    "J_恶意讨薪刑罚化："，
    "Direction_E："，
    "Direction_G："，
    "Direction_J："，
    "Overall_Direction："，
    "Mention_Labor_Rights"：，
    "Blame_Workers："，

示例一：

第一层编码：制度条款强度

| 编码项 | 数值 | 判读依据 |
| --- | --- | --- |
| Importance | 1 | 明确市级考核体系，具政策实施结构效力。 |
| E_保证金 | 2 | 明确将“工资保证金”作为考核内容，并要求“执行到位”，属于制度性安排。 |
| E_工资专用账户 | 2 | 明确列为考核项，并要求制度落实，原文：“工资专用账户和银行代发…执行的监督检查情况”。 |
| E_联合企业惩戒 | 2 | 提及“欠薪失信‘黑名单’认定及联动惩戒情况”，属于联合惩戒机制。 |
| G_周转金 | 0 | 未提及。 |
| G_工作考核办法 | 2 | 文件本身即为完整考核制度，涵盖流程、等级、问责与激励条款。 |
| J_司法回应性 | 2 | 涉及“行政司法联动打击…违法犯罪行为”，构成具体司法回应机制。 |
| J_欠薪刑罚化 | 1 | 涉及“拒不支付劳动报酬违法犯罪行为”，虽未细述条文，但具有刑事定性。 |
| J_恶意讨薪刑罚化 | 0 | 无对工人行为负面描写或刑事化内容。 |

第二层编码：政策方向性变项

| 指标名称 | 数值 | 判读依据 |
| --- | --- | --- |
| Direction_E | +1 | 明确强化企业制度执行，如“实名制管理”“工资保证金制度执行到位”等，强化责任落实。 |
| Direction_G | +1 | 明确政府主导考核制度、问责约谈、整改跟进等治理责任。 |
| Direction_J | +1 | 明确“行政司法联动”“追责”“依法查处”等回应机制。 |
| Overall_Direction | +1 | 各方向性均为正向，整体偏向保障劳工权益。 |

补充语义观察
- Mention_Labor_Rights：1（多次提及“保障农民工工资支付”“劳动报酬”等字眼）
- Blame_Workers：0（无指责语句）

示例二：

第一层编码：制度条款强度

| 编码项 | 数值 | 判读依据 |
| --- | --- | --- |
| Importance | 1 | 对保证金实施办法做出详细操作规定，具实质性规范效力。 |
| E_保证金 | 2 | 明确存储程序、差异化标准、减免条件、比例浮动机制等。 |
| E_工资专用账户 | 0 | 文件未涉及工资专用账户制度。 |
| E_联合企业惩戒 | 2 | 明确提及“拖欠农民工工资失信联合惩戒对象名单”，构成明确联合惩戒制度。 |
| G_周转金 | 0 | 未提及周转金制度。 |
| G_工作考核办法 | 0 | 未涉及考核制度或评估机制。 |
| J_司法回应性 | 0 | 未涉及司法援助、绿色通道或法院机制。 |
| J_欠薪刑罚化 | 0 | 虽有“行政处理或处罚”相关条款，但未涉及刑事责任。 |
| J_恶意讨薪刑罚化 | 0 | 无压制或负面描述维权行为的内容。 |

第二层编码：政策方向性变项

| 指标名称 | 数值 | 判读依据 |
| --- | --- | --- |
| Direction_E | +1 | 明确多项企业义务，如保函替代机制、差异化比例调整、补缴义务等，强化企业责任落实。 |
| Direction_G | +1 | 明确政府多级联动监管职责，如“监督检查”“动态名录管理”“限期补缴”等，属实质性治理承担。 |
| Direction_J | 0 | 文件未触及司法机制或法律援助内容。 |
| Overall_Direction | +1 | 政策体系严密，机制明晰，整体偏向保障劳工权益，方向性为正。 |

补充语义观察
- Mention_Labor_Rights：1  （提及“农民工工资”与保障支付责任）
- Blame_Workers：0 （无指责或负面描述工人行为）

示例三：

第一层编码：制度条款强度

| 编码项 | 数值 | 判读依据 |
| --- | --- | --- |
| Importance | 1 | 明确规范政府治理机制、考核制度，对政策结构与执行产生实际影响。 |
| E_保证金 | 2 | 明确将“工资保证金”制度列入考核内容，并要求实施到位。 |
| E_工资专用账户 | 2 | 多次强调农民工工资专用账户的实施情况及考核细则。 |
| E_联合企业惩戒 | 2 | 涉及“失信联合惩戒”“行政司法联动打击违法犯罪行为”，明确多部门联合惩戒机制。 |
| G_周转金 | 0 | 未提及周转金或应急资金制度。 |
| G_工作考核办法 | 2 | 文件本身即为考核办法，明确实施细则、等级标准及问责机制。 |
| J_司法回应性 | 2 | 涉及行政司法联动、“快立快审快结”等内容，说明司法机制明确设有回应机制。 |
| J_欠薪刑罚化 | 1 | 提及“打击拒不支付劳动报酬违法犯罪行为”，虽未详述刑罚机制，但明显涉及刑事责任。 |
| J_恶意讨薪刑罚化 | 0 | 完全未出现压制工人维权或负面指责语句。 |

第二层编码：政策方向性变项

| 指标名称 | 数值 | 判读依据 |
| --- | --- | --- |
| Direction_E | +1 | 文件多次强调制度实施到位、实名制、工资账户设立等企业义务落实，构成对企业责任的系统性强化。 |
| Direction_G | +1 | 市政府主导治理、设立议事协调机构、分级考核与问责机制。 |
| Direction_J | +1 | 明确行政司法联动、快速处理诉讼、畅通维权渠道。 |
| Overall_Direction | +1 | 三项方向性皆为正向，整体导向为保障劳工权益。 |

补充语义观察
- Mention_Labor_Rights：1  （多次提及“保障农民工工资支付”、“劳动报酬权益”、“社会公平正义”等字眼）
- Blame_Workers：0 （无指责或负面描述工人行为）

"""

    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "你是一位政策研究员，负责依据下列政策自动编码规则（Codebook），系统性阅读并判读中国地方政府针对“治理农民工欠薪”所发布的政策文件，并进行双层编码分析："},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message["content"].strip()

# 假設 df 已經載入，並且包含 'txt內容' 欄位
for i in range(len(df)):
    print(f"{i+1} / {len(df)}")  # 顯示進度，從 1 開始
    print("===================")

    text = df.at[i, 'txt內容']
    keywords = extract_keywords(text, model="o3-mini-2025-01-31")  # 這裡可以切換模型
    # gpt-4o
    # o1-2024-12-17
    # o3-mini-2025-01-31
    # 不支援o1-mini-2024-09-12

    df.at[i, 'o3-節錄內容'] = keywords

    print("原文:", text)
    print("節錄內容:", keywords)



1 / 7
原文:     【法宝引证码】 CLI.12.7550354
原文链接：https://www.pkulaw.com/lar/97fdb2695c1907da25a0cf3f11a51206bdfb.html
南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知(2024)
南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知
宁政办发〔2024〕29号
各区人民政府，江北新区管委会，市府各委办局，市各直属单位：
　　经市政府同意，现将《南京市保障农民工工资支付工作考核办法》印发给你们，请结合实际认真贯彻落实。
南京市人民政府办公厅
2024年5月29日
南京市保障农民工工资支付工作考核办法
　　第一条　为落实保障农民工工资支付工作的属地监管责任，有效预防和解决拖欠农民工工资问题，切实保障农民工劳动报酬权益，维护社会公平正义，促进社会和谐稳定，根据《保障农民工工资支付条例》《国务院办公厅关于印发保障农民工工资支付工作考核办法的通知》（国办发〔2023〕33号）和《省政府办公厅关于印发江苏省保障农民工工资支付工作考核办法的通知》（苏政办发〔2024〕13号）精神及有关规定，结合我市实际，制定本办法。
　　第二条　本办法适用于对各区人民政府（园区管委会）保障农民工工资支付工作的年度考核。
　　第三条　考核工作在市政府领导下，由市负责欠薪治理工作的议事协调机构（以下简称市议事协调机构）负责实施，市议事协调机构办公室（设在市人社局）具体组织落实，市议事协调机构成员单位根据职责分工共同做好考核工作。考核工作从2023年到2027年，每年开展一次。
　　第四条　考核工作坚持目标导向、问题导向、结果导向，坚持定性与定量评价相结合，遵循客观公正原则，突出重点，注重实效。
　　第五条　考核内容主要包括加强对保障农民工工资支付工作的组织领导、完善落实工资支付保障制度、治理欠薪特别是工程建设领域欠薪治理工作成效、人民群众满意度等情况。
　　第六条　加强组织领导主要考核内容为：各区人民政府（园区管委会）属地责任落实情况，工作协调机制建立运行情况，督查和问责制度落实情况，加强劳动保障监察执法能力建设等情况。
　　第七条　完善落实工资支付保障制度主要考核内容为：企业工资支付监控机制和欠薪预警系统建立运行情况，工程建设领域实名制管理、工资保证金、工资专用账

In [27]:
import re

text_column = 'o3-節錄內容'  # 存放文本的欄位

columns = [
    "省份",
    "地级市",
    "Importance",
    "E_保证金",
    "E_工资专用账户",
    "E_联合企业惩戒",
    "G_周转金",
    "G_工作考核办法",
    "J_司法回应性",
    "J_欠薪刑罚化",
    "J_恶意讨薪刑罚化",
    "Direction_E",
    "Direction_G",
    "Direction_J",
    "Overall_Direction",
    "Mention_Labor_Rights",
    "Blame_Workers"
]

# 建立正規表達式，兼容中英文冒號 [：:]
# 同時只擷取當行剩餘的內容 [^\r\n]+
pattern = r'(' + '|'.join(re.escape(col) for col in columns) + r')[：:]\s*([^\r\n]+)'

# ---- 1. 建立新的 DataFrame，先保留需要的原始欄位 ----
# 假設原始 df 有 "txt檔名"、"txt內容" 欄位要保留
df_extracted = df[["txt檔名", "txt內容", text_column]].copy()

# ---- 2. 新增要抽取的目標欄位，並初始為 None ----
for col in columns:
    df_extracted[col] = None

# ---- 3. 迴圈處理每一列，使用正則抽取指定欄位的值 ----
for idx, row in df.iterrows():
    content = str(row[text_column])  # 可能為 NaN 等，因此轉為字串

    # 若有需要可做這些替換：
    content = re.sub(r"Direction_E\s*（.*?）", "Direction_E", content)
    content = re.sub(r"Direction_G\s*（.*?）", "Direction_G", content)
    content = re.sub(r"Direction_J\s*（.*?）", "Direction_J", content)
    content = re.sub(r"Overall_Direction\s*（.*?）", "Overall_Direction", content)

    # 用 findall 抓取所有 (欄位名, 值) 配對
    matches = re.findall(pattern, content)

    # 填入 df_extracted 當中
    for col_name, val in matches:
        df_extracted.at[idx, col_name] = val.strip()

# 看抽取後的結果
df_extracted


,txt檔名,txt內容,o3-節錄內容,省份,地级市,Importance,E_保证金,E_工资专用账户,E_联合企业惩戒,G_周转金,G_工作考核办法,J_司法回应性,J_欠薪刑罚化,J_恶意讨薪刑罚化,Direction_E,Direction_G,Direction_J,Overall_Direction,Mention_Labor_Rights,Blame_Workers
0,240529_南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知.txt,【法宝引证码】 CLI.12.7550354\n原文链接：https://www.p...,省份：江苏 \n地级市：南京市 \n\nImportance：1 \n 判读依据：文...,江苏,南京市,1,2,2,2,0,2,2,1,0,+1,+1,+1,+1,1,0
1,180208_南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知.txt,【法宝引证码】 CLI.12.1437433\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据：...,江苏省,南京市,1,2,2,2,0,2,2,1,0,+1,+1,+1,+1,1,0
2,200210_南京市人民政府关于废止南京市建筑业施工企业民工工资保障金管理办法的决定.txt,【法宝引证码】 CLI.12.1437433\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 左证：“考...,江苏省,南京市,1,2,2,2,0,2,2,1,0,+1,+1,+1,+1,1,0
3,221027_h_南京市人力资源和社会保障局关于缓缴农民工工资保证金有关事项的通知.txt,【法宝引证码】 CLI.12.5628237\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据：该...,江苏省,南京市,1,2,0,0,0,0,0,0,0,-1,-1,0,-1,1,0
4,221206_h_南京市人力资源和社会保障局、南京市城乡建设委员会、南京市住房保障和房产局、...,【法宝引证码】 CLI.12.5684027\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据：本...,江苏省,南京市,1,2,2,0,0,0,0,0,0,+1,+1,0,+1,1,0
5,230213_h_南京市人力资源和社会保障局、南京市城乡建设委员会、南京市住房保障和房产局、...,【法宝引证码】 CLI.12.6442438\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 依据：本通...,江苏省,南京市,1,2,0,2,0,0,0,0,0,+1,+1,0,+1,1,0
6,170410_南京市政府办公厅关于全面治理拖欠农民工工资问题的实施意见.txt,【法宝引证码】 CLI.12.1268437\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据...,江苏省,南京市,1,2,2,2,2,2,2,2,0,+1,+1,+1,+1,1,0


In [28]:
# prompt: 將df輸出成xlsx

# 匯出成 Excel
output_file = "2_中國地方農民工欠薪治理文件與政策.xlsx"
df_extracted.to_excel(output_file, index=False) #index=False to avoid writing row index
print(f"已匯出結果至 {output_file}")

df_extracted

已匯出結果至 2_中國地方農民工欠薪治理文件與政策.xlsx


,txt檔名,txt內容,o3-節錄內容,省份,地级市,Importance,E_保证金,E_工资专用账户,E_联合企业惩戒,G_周转金,G_工作考核办法,J_司法回应性,J_欠薪刑罚化,J_恶意讨薪刑罚化,Direction_E,Direction_G,Direction_J,Overall_Direction,Mention_Labor_Rights,Blame_Workers
0,240529_南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知.txt,【法宝引证码】 CLI.12.7550354\n原文链接：https://www.p...,省份：江苏 \n地级市：南京市 \n\nImportance：1 \n 判读依据：文...,江苏,南京市,1,2,2,2,0,2,2,1,0,+1,+1,+1,+1,1,0
1,180208_南京市政府办公厅关于印发南京市保障农民工工资支付工作考核办法的通知.txt,【法宝引证码】 CLI.12.1437433\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据：...,江苏省,南京市,1,2,2,2,0,2,2,1,0,+1,+1,+1,+1,1,0
2,200210_南京市人民政府关于废止南京市建筑业施工企业民工工资保障金管理办法的决定.txt,【法宝引证码】 CLI.12.1437433\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 左证：“考...,江苏省,南京市,1,2,2,2,0,2,2,1,0,+1,+1,+1,+1,1,0
3,221027_h_南京市人力资源和社会保障局关于缓缴农民工工资保证金有关事项的通知.txt,【法宝引证码】 CLI.12.5628237\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据：该...,江苏省,南京市,1,2,0,0,0,0,0,0,0,-1,-1,0,-1,1,0
4,221206_h_南京市人力资源和社会保障局、南京市城乡建设委员会、南京市住房保障和房产局、...,【法宝引证码】 CLI.12.5684027\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据：本...,江苏省,南京市,1,2,2,0,0,0,0,0,0,+1,+1,0,+1,1,0
5,230213_h_南京市人力资源和社会保障局、南京市城乡建设委员会、南京市住房保障和房产局、...,【法宝引证码】 CLI.12.6442438\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 依据：本通...,江苏省,南京市,1,2,0,2,0,0,0,0,0,+1,+1,0,+1,1,0
6,170410_南京市政府办公厅关于全面治理拖欠农民工工资问题的实施意见.txt,【法宝引证码】 CLI.12.1268437\n原文链接：https://www.p...,省份：江苏省 \n地级市：南京市 \n\nImportance：1 \n 判读依据...,江苏省,南京市,1,2,2,2,2,2,2,2,0,+1,+1,+1,+1,1,0
